In [ ]:
#This notebook filters all the gas stations located inside Mexico City using an 
# shp. file and gas prices provided by the CRE.

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import earthpy as et
from shapely.geometry import Point, Polygon

In [3]:
def obten_gasolineras(poligono,gasolineras):
    gasolineras_dentro_poligono = []
    for i in range(len(gasolineras)):
        punto = Point(gasolineras.loc[i,"Longitud"],gasolineras.loc[i,"Latitud"])
        if punto.within(poligono) == True:
            gasolineras_dentro_poligono.append(gasolineras.iloc[i])
    
    return gasolineras_dentro_poligono

In [6]:
mapa = gpd.read_file('México_Estados.shp')
gasolineras = pd.read_excel('precios_por_estacion.xlsx')

In [33]:
precios_mexico = pd.DataFrame(index=range(1),columns=['place_id','name','cre_id','Longitud','Latitud','regular','premium'])

In [34]:
for i in range(len(mapa)):
    precios_locales = obten_gasolineras(mapa.loc[i]['geometry'],gasolineras)
    nombre_estado = mapa.loc[i]['ESTADO']
    precios_locales_df = pd.DataFrame(precios_locales)
    precios_locales_df['Estado'] = mapa.loc[i]['ESTADO']
    precios_locales_df = precios_locales_df.drop(['Unnamed: 0'],axis=1)
    precios_locales_df['regular'] = precios_locales_df['regular'].replace(0,precios_locales_df['regular'].median())
    precios_locales_df['premium'] = precios_locales_df['premium'].replace([0,0.01,1],precios_locales_df['premium'].median())
    precios_mexico = pd.concat([precios_mexico,precios_locales_df])

In [35]:
precios_mexico = precios_mexico.drop(0)

In [36]:
precios_mexico.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12476 entries, 5 to 12632
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   place_id  12476 non-null  object 
 1   name      12476 non-null  object 
 2   cre_id    12476 non-null  object 
 3   Longitud  12476 non-null  float64
 4   Latitud   12476 non-null  float64
 5   regular   12476 non-null  float64
 6   premium   12476 non-null  float64
 7   Estado    12476 non-null  object 
dtypes: float64(4), object(4)
memory usage: 877.2+ KB


In [41]:
precios_mexico.head()

place_id                                name              cre_id  Longitud  \
5      2044   ESTACION RAEL, S. DE R.L. DE C.V.  PL/842/EXP/ES/2015 -117.0715   
8      2047   ESTACION RAEL, S. DE R.L. DE C.V.  PL/860/EXP/ES/2015 -116.6079   
14     2053           CIRCULO DOS, S.A. DE C.V.  PL/635/EXP/ES/2015 -117.0271   
23     2062         Becktrop Operadora SA de CV  PL/708/EXP/ES/2015 -117.0249   
25     2064  ESTACION LA VICTORIA, S.A. DE C.V.  PL/636/EXP/ES/2015 -117.0466   

     Latitud  regular  premium           Estado  
5   32.51342    19.99    22.79  Baja California  
8   31.86546    20.49    22.49  Baja California  
14  32.52790    20.39    22.99  Baja California  
23  32.45102    19.39    22.49  Baja California  
25  32.53171    19.99    22.99  Baja California

In [40]:
precios_mexico.to_excel('precios_gasolinas_mx.xlsx')